In [2]:
!pip install librosa matplotlib spafe torch pandas

  Using cached torch-2.1.0-cp38-cp38-manylinux1_x86_64.whl (670.2 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.4/173.4 KB 447.2 kB/s eta 0:00:00a 0:00:01
  Using cached networkx-3.1-py3-none-any.whl (2.1 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 310.3 kB/s eta 0:00:0000:0100:02
  Using cached filelock-3.12.4-py3-none-any.whl (11 kB)
  Using cached nvidia_nccl_cu12-2.18.1-py3-none-manylinux1_x86_64.whl (209.8 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 404.8 kB/s eta 0:00:0000:0100:25
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 415.2 kB/s eta 0:00:0000:0100:05
     ━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
import librosa
import librosa.display
import numpy as np
import IPython.display as ipd
import matplotlib.pyplot as plt
%matplotlib inline

from collections import defaultdict
from spafe.utils import vis
from spafe.features.lfcc import lfcc
import pandas as pd

In [ ]:
import torch
import torch.nn as nn
import math

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class SelfAttention(nn.Module):

  def __init__(self, embed_size, num_heads):

    super(SelfAttention, self).__init__()

    self.embed_size = embed_size
    self.num_heads = num_heads
    self.head_dim = embed_size // num_heads

    assert (self.head_dim * num_heads == embed_size), "Embed size needs to be divisible by heads"

    self.values = nn.Linear(self.head_dim, self.head_dim, bias = False)
    self.keys = nn.Linear(self.head_dim, self.head_dim, bias = False)
    self.queries = nn.Linear(self.head_dim, self.head_dim, bias = False)
    self.fc_out = nn.Linear(num_heads * self.head_dim, embed_size)


  def forward(self, values, keys, query, mask):
    N = query.shape[0]
    value_len, key_len, query_len = values.shape[1], keys.shape[1], query.shape[1]

    # Split embedding into self.num_heads pieces
    values = values.reshape(N, value_len, self.num_heads, self.head_dim)
    print("values:", values.shape)
    keys = keys.reshape(N, key_len, self.num_heads, self.head_dim)
    print("keys:", keys.shape)
    queries = query.reshape(N, query_len, self.num_heads, self.head_dim)
    print("queries:", query.shape)


    values = self.values(values)
    keys = self.keys(keys)
    queries = self.queries(queries)
    energy = torch.einsum("nqhd,nkhd->nhqk", [queries, keys]) # MatMul Q and K
    # queries shape: (N, query_len, heads, heads_dim)
    # keys shape: (N, query_len, heads, heads_dim)
    # energy shape: (N, heads, query_len, key_len)

    if mask is not None:
      energy = energy.masked_fill(mask == 0, float("-1e20"))

    attention = torch.softmax(energy / (self.embed_size ** 0.5), dim = 3)

    out = torch.einsum("nhql,nlhd->nqhd", [attention, values]).reshape(
      N, query_len, self.num_heads * self.head_dim
    )

    out = self.fc_out(out)

    return out

In [ ]:
class PositionalEncoding(nn.Module):

  def __init__(self, embed_size, dropout, max_len = 5000):
    super().__init__()

    self.dropout = nn.Dropout(p = dropout)

    position = torch.arange(max_len).unsqueeze(1)
    div_term = torch.exp(torch.arange(0, embed_size, 2) * (-math.log(10000.0) / embed_size))
    self.position_encoding = torch.zeros(max_len, 1, embed_size).to(device)
    self.position_encoding[:, 0, 0::2] = torch.sin(position * div_term).to(device)
    self.position_encoding[:, 0, 1::2] = torch.cos(position * div_term).to(device)
    self.register_buffer('pe', self.position_encoding)

  def forward(self, x):
    x = x + self.position_encoding[:x.size(0)]
    return self.dropout(x)

In [ ]:
class TransformerBlock(nn.Module):

  def __init__(self, embed_size, heads, dropout, forward_expansion):
    super(TransformerBlock, self).__init__()

    self.attention = SelfAttention(embed_size, heads)
    self.norm1 = nn.LayerNorm(embed_size)
    self.norm2 = nn.LayerNorm(embed_size)

    self.feed_forward = nn.Sequential(
      nn.Linear(embed_size, forward_expansion * embed_size),
      nn.ReLU(),
      nn.Linear(forward_expansion * embed_size, embed_size)
    )

    self.dropout = nn.Dropout(dropout)


  def forward(self, value, key, query, mask):
    attention = self.attention(value, key, query, mask)

    x = self.dropout(self.norm1((attention + query)))

    forward = self.feed_forward(x)
    out = self.dropout(self.norm2(x + forward))

    return out

In [ ]:
class Encoder(nn.Module):

  def __init__(
    self,
    src_vocab_size,
    embed_size,
    num_layers,
    heads,
    device,
    forward_expansion,
    dropout,
    max_length
  ):
    super(Encoder, self).__init__()
    self.embed_size = embed_size
    self.device = device
    self.position_embedding = PositionalEncoding(embed_size, dropout, src_vocab_size)

    self.layers = nn.ModuleList(
      [
        TransformerBlock(embed_size, heads, dropout, forward_expansion) for _ in range(num_layers)
      ]
    )

    self.dropout = nn.Dropout(dropout)


  def forward(self, x, mask):
    print(x.shape)
    N, seq_length = x.shape

    positions = torch.arange(0, seq_length).expand(N, seq_length).to(self.device)

    out = self.dropout(self.position_embedding(positions))

    for layer in self.layers:
      out = layer(out, out, out, mask)

    return out

In [ ]:
class Transformer(nn.Module):

  def __init__(
    self,
    src_vocab_size,
    src_pad_index,
    embed_size = 256,
    num_layers = 6,
    forward_expansion = 4,
    heads = 8,
    dropout = 0,
    device = "cuda",
    max_length = 500
  ):
    super(Transformer, self).__init__()

    self.encoder = Encoder(
      src_vocab_size, embed_size, num_layers, heads,
      device, forward_expansion, dropout, max_length)

    self.src_pad_index = src_pad_index
    self.device = device


  def make_src_mask(self, src):
    src_mask = (src != self.src_pad_index).unsqueeze(1).unsqueeze(2)

    # (N, 1, 1, src_len)
    return src_mask.to(self.device)


  def forward(self, src):
    src_mask = self.make_src_mask(src)
    encoder_output = self.encoder(src, src_mask)

    return encoder_output

In [ ]:
from torch.utils.data import random_split, Dataset, DataLoader

# class AudioFeatureDataset2(Dataset):
#     def __init__(self, annotations_file, mode = 'train'):
#         self.features = pd.read_csv(annotations_file)
#         self.features = self.features.drop(['Unnamed: 0'], axis = 1)
#         # splitting the test and validation dataset in (80%,20%)
#         self.train_subset, self.valid_subset = random_split(self.features, [0.8, 0.2])
#         # defining the x_train and y_train(target == "class")
#         self.x_train = torch.Tensor(self.train_subset.dataset.drop(['class', str(len(self.train_subset.dataset.columns) - 2)], axis = 1).values)
#         self.y_train = torch.Tensor(self.train_subset.dataset['class'].values)
#         # defining the x_valid and y_valid(target == "class")
#         self.x_valid = torch.Tensor(self.valid_subset.dataset.drop(['class', str(len(self.train_subset.dataset.columns) - 2)], axis = 1).values)
#         self.y_valid = torch.Tensor(self.valid_subset.dataset['class'].values)
#         self.mode = mode
#         print("x_train: " , self.x_train.shape, " x_valid:", self.x_valid.shape  )

#     def __len__(self):
#         if self.mode == 'train':
#             return self.x_train.shape[0]

#         return self.x_valid.shape[0]

#     def __getitem__(self, idx):
#         if self.mode == 'train':
#             return self.x_train[idx], self.y_train[idx]

#         return self.x_valid[idx], self.y_valid[idx]

In [ ]:
cd /content/drive/MyDrive/DL_course/features

[Errno 2] No such file or directory: '/content/drive/MyDrive/DL_course/features'
/content


In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader, random_split

class AudioFeatureDataset(Dataset):
    def __init__(self, annotations_file, mode='train'):
        self.data = pd.read_csv(annotations_file)
        self.data = self.data.drop(['Unnamed: 0'], axis=1)

        # Splitting the dataset into train and validation sets
        total_samples = len(self.data)
        train_size = int(0.8 * total_samples)
        valid_size = total_samples - train_size

        if mode == 'train':
            self.data = self.data.iloc[:train_size]
        else:
            self.data = self.data.iloc[train_size:]

        self.features = torch.Tensor(self.data.drop(['0', 'class'], axis=1).values)
        self.labels = torch.Tensor(self.data['class'].values)
        self.mode = mode

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

# Example usage
dataset = AudioFeatureDataset('./2075_concatenate_dm_4featu.csv', mode='train')
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

for batch in dataloader:
    x, y = batch
    # Your training code here
x_train = dataset[0][0]
x_target = dataset[0][1]

print(len(dataset))
print(len(dataset[0][0]))

dataset_val = AudioFeatureDataset('./2075_concatenate_dm_4featu.csv',mode="val")
dataloader_val = DataLoader(dataset, batch_size=64, shuffle=True)
print(len(dataset_val))

FileNotFoundError: ignored

In [ ]:
import torch

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
num_layers = 4
src_vocab_size = 1
src_pad_index = 0
embed_size = 296
num_heads = 4
dropout = 0.1
output_size = 1
batch_size = 64

model = Transformer(
  src_vocab_size,
  src_pad_index,
  embed_size = embed_size,
  dropout = dropout,
  heads = num_heads,
  num_layers = num_layers,
  ).to(device)

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr = 0.000003)

loss_fn = nn.MSELoss()

running_loss = 0.
last_loss = 0.

model.train()

epochs = 1000

for epoch in range(epochs):
  print(f"Epoch {epoch+1}\n-------------------------------")
  for batch, (X, y) in enumerate(dataloader):
    # Compute prediction and loss
    X = X.to(device)
    y = y.to(device)
    # print(X.shape)
    # break
    pred = model(X)
    loss = loss_fn(pred, y)

    # Backpropagation
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    if batch % 100 == 0:
        loss, current = loss.item(), (batch + 1) * len(X)
        print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


  model.eval()
  size = len(dataloader_val.dataset)
  num_batches = len(dataloader_val)
  test_loss, correct = 0, 0

  with torch.no_grad():
    for X in dataloader_val:
        pred = model(X)
        test_loss += loss_fn(pred, y).item()
        correct += (pred.argmax(1) == y).type(torch.float).sum().item()

  test_loss /= num_batches
  correct /= size
  print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

print('Done')